In [1]:
# We only need the features extracted for the final 7 images (sub-images) in the sequence

In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.transform import resize

2025-09-08 21:09:27.701245: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-08 21:09:27.737180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-08 21:09:27.737203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-08 21:09:27.738105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-08 21:09:27.744044: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# import the trained model

model = tf.keras.models.load_model("../../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5")

# trained_gmp_model_dense_32_new
# trained_gmp_model_dense_32_new

2025-09-08 21:10:57.411371: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [4]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [5]:
# Define the feature extractor model

# feature extractor input
feat_ext_input = model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [6]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [7]:
# Okay, now let's worry about the data. We only need the features extracted for the last 7 images in teh sequence. Let's get these here? Also note that we do not have anything to do with the density maps for now, for the stage 1 implementaiton at least.

In [8]:
# The following functions will help create all the required matrices

In [9]:
def chose_xml_and_jpeg(file_location):
    # list all files in location
    list_of_all_files = os.listdir(file_location)
    # sort files
    list_of_all_files.sort()
    # separate xml and jpeg files
    all_xml_files = [file for file in list_of_all_files if file.split('.')[-1] == 'xml']
    all_xml_files.sort()
    all_jpeg_files = [file for file in list_of_all_files if file not in all_xml_files]
    all_jpeg_files.sort()
    # get the final 7 image files
    chosen_jpeg_files = all_jpeg_files[-7:]
    
    return chosen_jpeg_files

In [10]:
def create_sub_windows(folder_path, file, stride = 24, kernel_size = 300):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [11]:
def extract_features(file_name, file_path):
    # get the subwindows
    subwindows = create_sub_windows(file_path, file_name, 24, 300)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    
    return extracted_featrues

In [12]:
# Let's first work on the train data blocks

In [13]:
# block 0101

In [14]:
block_0101 = "../../../Spring_2024/S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml"

In [15]:
images_block_0101 = chose_xml_and_jpeg(block_0101)

In [16]:
images_block_0101

['Block0101_2020_08_26.jpeg',
 'Block0101_2020_08_27.jpeg',
 'Block0101_2020_08_28.jpeg',
 'Block0101_2020_08_31.jpeg',
 'Block0101_2020_09_02.jpeg',
 'Block0101_2020_09_07.jpeg',
 'Block0101_2020_09_16.jpeg']

In [17]:
%%time
# get the extracted input features

blk_0101_all_features = []
for file in images_block_0101:
    extracted_features = extract_features(file, block_0101)
    blk_0101_all_features.append(extracted_features)

(1376, 300, 300, 3)


2025-09-08 19:43:36.052080: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


43/43 [==============================] - 4s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 17ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 17ms/step
CPU times: user 2min 34s, sys: 13.5 s, total: 2min 47s
Wall time: 2min 46s


In [18]:
# stack the extracted features
blk_0101_stacked_extracted_features = np.stack(blk_0101_all_features, axis = 1)

In [19]:
blk_0101_stacked_extracted_features.shape

(1376, 7, 32)

In [20]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0101_stacked_extracted_features)

0.20483392

In [21]:
# let's not save or stack this for now, let's stack all train data in order, and then save

In [22]:
# block 0102

In [23]:
block_0102 = "../../../Spring_2024/S_lab_TasselNet/Block_2_TN/Block_2_images_and_xml"

In [24]:
images_block_0102 = chose_xml_and_jpeg(block_0102)

In [25]:
images_block_0102

['Block0102_2020_08_26.jpeg',
 'Block0102_2020_08_27.jpeg',
 'Block0102_2020_08_28.jpeg',
 'Block0102_2020_08_31.jpeg',
 'Block0102_2020_09_02.jpeg',
 'Block0102_2020_09_07.jpeg',
 'Block0102_2020_09_16.jpeg']

In [26]:
%%time
# get the extracted input features

blk_0102_all_features = []
for file in images_block_0102:
    extracted_features = extract_features(file, block_0102)
    blk_0102_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 17ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 17ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 34s, sys: 12.2 s, total: 2min 46s
Wall time: 2min 43s


In [27]:
# stack the extracted features
blk_0102_stacked_extracted_features = np.stack(blk_0102_all_features, axis = 1)

In [28]:
blk_0102_stacked_extracted_features.shape

(1376, 7, 32)

In [29]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0102_stacked_extracted_features)

0.21673495

In [30]:
# block 0203

In [31]:
block_0203 = "../../../Spring_2024/S_lab_TasselNet/Block_9_TN/Block_9_images_and_xml"

In [32]:
images_block_0203 = chose_xml_and_jpeg(block_0203)

In [33]:
images_block_0203

['Block0203_2020_08_26.jpeg',
 'Block0203_2020_08_27.jpeg',
 'Block0203_2020_08_28.jpeg',
 'Block0203_2020_08_31.jpeg',
 'Block0203_2020_09_02.jpeg',
 'Block0203_2020_09_07.jpeg',
 'Block0203_2020_09_16.jpeg']

In [34]:
%%time
# get the extracted input features

blk_0203_all_features = []
for file in images_block_0203:
    extracted_features = extract_features(file, block_0203)
    blk_0203_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.4 s, total: 2min 47s
Wall time: 2min 45s


In [35]:
# stack the extracted features
blk_0203_stacked_extracted_features = np.stack(blk_0203_all_features, axis = 1)

In [36]:
blk_0203_stacked_extracted_features.shape

(1376, 7, 32)

In [37]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0203_stacked_extracted_features)

0.28086844

In [38]:
# block 0301

In [39]:
block_0301 = "../../../Spring_2024/S_lab_TasselNet/Block_13_TN/Block_13_images_and_xml"

In [40]:
images_block_0301 = chose_xml_and_jpeg(block_0301)

In [41]:
images_block_0301

['Block0301_2020_08_26.jpeg',
 'Block0301_2020_08_27.jpeg',
 'Block0301_2020_08_28.jpeg',
 'Block0301_2020_08_31.jpeg',
 'Block0301_2020_09_02.jpeg',
 'Block0301_2020_09_07.jpeg',
 'Block0301_2020_09_16.jpeg']

In [42]:
%%time
# get the extracted input features

blk_0301_all_features = []
for file in images_block_0301:
    extracted_features = extract_features(file, block_0301)
    blk_0301_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 19ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 19ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 34s, sys: 11 s, total: 2min 45s
Wall time: 2min 43s


In [43]:
# stack the extracted features
blk_0301_stacked_extracted_features = np.stack(blk_0301_all_features, axis = 1)

In [44]:
blk_0301_stacked_extracted_features.shape

(1376, 7, 32)

In [45]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0301_stacked_extracted_features)

0.23826104

In [46]:
# We can stack all these now and save?

all_train_targets_list = [blk_0101_stacked_extracted_features, blk_0102_stacked_extracted_features, blk_0203_stacked_extracted_features, blk_0301_stacked_extracted_features]

In [47]:
# stack all these together?
all_train_targets = np.vstack(all_train_targets_list)

In [48]:
all_train_targets.shape

(5504, 7, 32)

In [49]:
# sanity check
print(np.mean(all_train_targets[1376*0:1376*1, :, :] == blk_0101_stacked_extracted_features))
print(np.mean(all_train_targets[1376*1:1376*2, :, :] == blk_0102_stacked_extracted_features))
print(np.mean(all_train_targets[1376*2:1376*3, :, :] == blk_0203_stacked_extracted_features))
print(np.mean(all_train_targets[1376*3:1376*4, :, :] == blk_0301_stacked_extracted_features))

1.0
1.0
1.0
1.0


In [50]:
# save the stack
np.save("data/train_out_targets/all_train_targets.npy", all_train_targets)

In [51]:
# sanity check
np.mean(all_train_targets == np.load("data/train_out_targets/all_train_targets.npy"))

1.0

In [52]:
# Do this also for the validation data, and then stop for the day?

In [53]:
# block 0204

In [54]:
block_0204 = "../../../Spring_2024/S_lab_TasselNet/Block_10_TN/Block_10_images_and_xml"

In [55]:
images_block_0204 = chose_xml_and_jpeg(block_0204)

In [56]:
images_block_0204

['Block0204_2020_08_26.jpeg',
 'Block0204_2020_08_27.jpeg',
 'Block0204_2020_08_28.jpeg',
 'Block0204_2020_08_31.jpeg',
 'Block0204_2020_09_02.jpeg',
 'Block0204_2020_09_07.jpeg',
 'Block0204_2020_09_16.jpeg']

In [57]:
%%time
# get the extracted input features

blk_0204_all_features = []
for file in images_block_0204:
    extracted_features = extract_features(file, block_0204)
    blk_0204_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 19ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 11.5 s, total: 2min 46s
Wall time: 2min 44s


In [58]:
# stack the extracted features
blk_0204_stacked_extracted_features = np.stack(blk_0204_all_features, axis = 1)

In [59]:
blk_0204_stacked_extracted_features.shape

(1376, 7, 32)

In [60]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0204_stacked_extracted_features)

0.24017075

In [61]:
# save the data
np.save("data/valid_out_targets/all_valid_targets.npy", blk_0204_stacked_extracted_features)

In [62]:
# sanity check
np.mean(blk_0204_stacked_extracted_features == np.load("data/valid_out_targets/all_valid_targets.npy"))

1.0

Preprocessing for the test data

In [63]:
# Block 0103

In [64]:
block_0103 = "../../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml"

In [65]:
images_block_0103 = chose_xml_and_jpeg(block_0103)

In [66]:
images_block_0103

['Block0103_2020_08_26.jpeg',
 'Block0103_2020_08_27.jpeg',
 'Block0103_2020_08_28.jpeg',
 'Block0103_2020_08_31.jpeg',
 'Block0103_2020_09_02.jpeg',
 'Block0103_2020_09_07.jpeg',
 'Block0103_2020_09_16.jpeg']

In [67]:
%%time
# get the extracted input features

blk_0103_all_features = []
for file in images_block_0103:
    extracted_features = extract_features(file, block_0103)
    blk_0103_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 34s, sys: 11.6 s, total: 2min 46s
Wall time: 2min 43s


In [68]:
# stack the extracted features
blk_0103_stacked_extracted_features = np.stack(blk_0103_all_features, axis = 1)

In [69]:
blk_0103_stacked_extracted_features.shape

(1376, 7, 32)

In [70]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0103_stacked_extracted_features)

0.16587366

In [71]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0103.npy", blk_0103_stacked_extracted_features)

In [72]:
# sanity check
np.mean(blk_0103_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0103.npy"))

1.0

In [73]:
# Block 0104

In [74]:
block_0104 = "../../../Spring_2024/S_lab_TasselNet/Block_4_TN/Block_4_images_and_xml"

In [75]:
images_block_0104 = chose_xml_and_jpeg(block_0104)

In [76]:
images_block_0104

['Block0104_2020_08_26.jpeg',
 'Block0104_2020_08_27.jpeg',
 'Block0104_2020_08_28.jpeg',
 'Block0104_2020_08_31.jpeg',
 'Block0104_2020_09_02.jpeg',
 'Block0104_2020_09_07.jpeg',
 'Block0104_2020_09_16.jpeg']

In [77]:
%%time
# get the extracted input features

blk_0104_all_features = []
for file in images_block_0104:
    extracted_features = extract_features(file, block_0104)
    blk_0104_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 11.6 s, total: 2min 47s
Wall time: 2min 44s


In [78]:
# stack the extracted features
blk_0104_stacked_extracted_features = np.stack(blk_0104_all_features, axis = 1)

In [79]:
blk_0104_stacked_extracted_features.shape

(1376, 7, 32)

In [80]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0104_stacked_extracted_features)

0.23606355

In [81]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0104.npy", blk_0104_stacked_extracted_features)

In [82]:
# sanity check
np.mean(blk_0104_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0104.npy"))

1.0

In [83]:
# Block 0105

In [84]:
block_0105 = "../../../Spring_2024/S_lab_TasselNet/Block_5_TN/Block_5_images_and_xml"

In [85]:
images_block_0105 = chose_xml_and_jpeg(block_0105)

In [86]:
images_block_0105

['Block0105_2020_08_26.jpeg',
 'Block0105_2020_08_27.jpeg',
 'Block0105_2020_08_28.jpeg',
 'Block0105_2020_08_31.jpeg',
 'Block0105_2020_09_02.jpeg',
 'Block0105_2020_09_07.jpeg',
 'Block0105_2020_09_16.jpeg']

In [87]:
%%time
# get the extracted input features

blk_0105_all_features = []
for file in images_block_0105:
    extracted_features = extract_features(file, block_0105)
    blk_0105_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.1 s, total: 2min 47s
Wall time: 2min 44s


In [88]:
# stack the extracted features
blk_0105_stacked_extracted_features = np.stack(blk_0105_all_features, axis = 1)

In [89]:
blk_0105_stacked_extracted_features.shape

(1376, 7, 32)

In [90]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0105_stacked_extracted_features)

0.25614625

In [91]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0105.npy", blk_0105_stacked_extracted_features)

In [92]:
# sanity check
np.mean(blk_0105_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0105.npy"))

1.0

In [93]:
# Block 0106

In [94]:
block_0106 = "../../../Spring_2024/S_lab_TasselNet/Block_6_TN/Block_6_images_and_xml"

In [95]:
images_block_0106 = chose_xml_and_jpeg(block_0106)

In [96]:
images_block_0106

['Block0106_2020_08_26.jpeg',
 'Block0106_2020_08_27.jpeg',
 'Block0106_2020_08_28.jpeg',
 'Block0106_2020_08_31.jpeg',
 'Block0106_2020_09_02.jpeg',
 'Block0106_2020_09_07.jpeg',
 'Block0106_2020_09_16.jpeg']

In [97]:
%%time
# get the extracted input features

blk_0106_all_features = []
for file in images_block_0106:
    extracted_features = extract_features(file, block_0106)
    blk_0106_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 11.7 s, total: 2min 47s
Wall time: 2min 44s


In [98]:
# stack the extracted features
blk_0106_stacked_extracted_features = np.stack(blk_0106_all_features, axis = 1)

In [99]:
blk_0106_stacked_extracted_features.shape

(1376, 7, 32)

In [100]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0106_stacked_extracted_features)

0.33636108

In [101]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0106.npy", blk_0106_stacked_extracted_features)

In [102]:
# sanity check
np.mean(blk_0106_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0106.npy"))

1.0

In [103]:
# Block 0201

In [104]:
block_0201 = "../../../Spring_2024/S_lab_TasselNet/Block_7_TN/Block_7_images_and_xml"

In [105]:
images_block_0201 = chose_xml_and_jpeg(block_0201)

In [106]:
images_block_0201

['Block0201_2020_08_26.jpeg',
 'Block0201_2020_08_27.jpeg',
 'Block0201_2020_08_28.jpeg',
 'Block0201_2020_08_31.jpeg',
 'Block0201_2020_09_02.jpeg',
 'Block0201_2020_09_07.jpeg',
 'Block0201_2020_09_16.jpeg']

In [107]:
%%time
# get the extracted input features

blk_0201_all_features = []
for file in images_block_0201:
    extracted_features = extract_features(file, block_0201)
    blk_0201_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 36s, sys: 12.3 s, total: 2min 48s
Wall time: 2min 45s


In [108]:
# stack the extracted features
blk_0201_stacked_extracted_features = np.stack(blk_0201_all_features, axis = 1)

In [109]:
blk_0201_stacked_extracted_features.shape

(1376, 7, 32)

In [110]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0201_stacked_extracted_features)

0.24373145

In [111]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0201.npy", blk_0201_stacked_extracted_features)

In [112]:
# sanity check
np.mean(blk_0201_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0201.npy"))

1.0

In [113]:
# Block 0202

In [114]:
block_0202 = "../../../Spring_2024/S_lab_TasselNet/Block_8_TN/Block_8_images_and_xml"

In [115]:
images_block_0202 = chose_xml_and_jpeg(block_0202)

In [116]:
images_block_0202

['Block0202_2020_08_26.jpeg',
 'Block0202_2020_08_27.jpeg',
 'Block0202_2020_08_28.jpeg',
 'Block0202_2020_08_31.jpeg',
 'Block0202_2020_09_02.jpeg',
 'Block0202_2020_09_07.jpeg',
 'Block0202_2020_09_16.jpeg']

In [117]:
%%time
# get the extracted input features

blk_0202_all_features = []
for file in images_block_0202:
    extracted_features = extract_features(file, block_0202)
    blk_0202_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 36s, sys: 12.9 s, total: 2min 49s
Wall time: 2min 46s


In [118]:
# stack the extracted features
blk_0202_stacked_extracted_features = np.stack(blk_0202_all_features, axis = 1)

In [119]:
blk_0202_stacked_extracted_features.shape

(1376, 7, 32)

In [120]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0202_stacked_extracted_features)

0.21695612

In [121]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0202.npy", blk_0202_stacked_extracted_features)

In [122]:
# sanity check
np.mean(blk_0202_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0202.npy"))

1.0

In [123]:
# Block 0205

In [124]:
block_0205 = "../../../Spring_2024/S_lab_TasselNet/Block_11_TN/Block_11_images_and_xml"

In [125]:
images_block_0205 = chose_xml_and_jpeg(block_0205)

In [126]:
images_block_0205

['Block0205_2020_08_26.jpeg',
 'Block0205_2020_08_27.jpeg',
 'Block0205_2020_08_28.jpeg',
 'Block0205_2020_08_31.jpeg',
 'Block0205_2020_09_02.jpeg',
 'Block0205_2020_09_07.jpeg',
 'Block0205_2020_09_16.jpeg']

In [127]:
%%time
# get the extracted input features

blk_0205_all_features = []
for file in images_block_0205:
    extracted_features = extract_features(file, block_0205)
    blk_0205_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 17ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.3 s, total: 2min 47s
Wall time: 2min 45s


In [128]:
# stack the extracted features
blk_0205_stacked_extracted_features = np.stack(blk_0205_all_features, axis = 1)

In [129]:
blk_0205_stacked_extracted_features.shape

(1376, 7, 32)

In [130]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0205_stacked_extracted_features)

0.32147333

In [131]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0205.npy", blk_0205_stacked_extracted_features)

In [132]:
# sanity check
np.mean(blk_0205_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0205.npy"))

1.0

In [133]:
# Block 0206

In [134]:
block_0206 = "../../../Spring_2024/S_lab_TasselNet/Block_12_TN/Block_12_images_and_xml"

In [135]:
images_block_0206 = chose_xml_and_jpeg(block_0206)

In [136]:
images_block_0206

['Block0206_2020_08_26.jpeg',
 'Block0206_2020_08_27.jpeg',
 'Block0206_2020_08_28.jpeg',
 'Block0206_2020_08_31.jpeg',
 'Block0206_2020_09_02.jpeg',
 'Block0206_2020_09_07.jpeg',
 'Block0206_2020_09_16.jpeg']

In [137]:
%%time
# get the extracted input features

blk_0206_all_features = []
for file in images_block_0206:
    extracted_features = extract_features(file, block_0206)
    blk_0206_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.4 s, total: 2min 47s
Wall time: 2min 45s


In [138]:
# stack the extracted features
blk_0206_stacked_extracted_features = np.stack(blk_0206_all_features, axis = 1)

In [139]:
blk_0206_stacked_extracted_features.shape

(1376, 7, 32)

In [140]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0206_stacked_extracted_features)

0.28877684

In [141]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0206.npy", blk_0206_stacked_extracted_features)

In [142]:
# sanity check
np.mean(blk_0206_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0206.npy"))

1.0

In [143]:
# Block 0302

In [144]:
block_0302 = "../../../Spring_2024/S_lab_TasselNet/Block_14_TN/Block_14_images_and_xml"

In [145]:
images_block_0302 = chose_xml_and_jpeg(block_0302)

In [146]:
images_block_0302

['Block0302_2020_08_26.jpeg',
 'Block0302_2020_08_27.jpeg',
 'Block0302_2020_08_28.jpeg',
 'Block0302_2020_08_31.jpeg',
 'Block0302_2020_09_02.jpeg',
 'Block0302_2020_09_07.jpeg',
 'Block0302_2020_09_16.jpeg']

In [147]:
%%time
# get the extracted input features

blk_0302_all_features = []
for file in images_block_0302:
    extracted_features = extract_features(file, block_0302)
    blk_0302_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.3 s, total: 2min 48s
Wall time: 2min 45s


In [148]:
# stack the extracted features
blk_0302_stacked_extracted_features = np.stack(blk_0302_all_features, axis = 1)

In [149]:
blk_0302_stacked_extracted_features.shape

(1376, 7, 32)

In [150]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0302_stacked_extracted_features)

0.25968924

In [151]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0302.npy", blk_0302_stacked_extracted_features)

In [152]:
# sanity check
np.mean(blk_0302_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0302.npy"))

1.0

In [153]:
# Block 0303

In [154]:
block_0303 = "../../../Spring_2024/S_lab_TasselNet/Block_15_TN/Block_15_images_and_xml"

In [155]:
images_block_0303 = chose_xml_and_jpeg(block_0303)

In [156]:
images_block_0303

['Block0303_2020_08_26.jpeg',
 'Block0303_2020_08_27.jpeg',
 'Block0303_2020_08_28.jpeg',
 'Block0303_2020_08_31.jpeg',
 'Block0303_2020_09_02.jpeg',
 'Block0303_2020_09_07.jpeg',
 'Block0303_2020_09_16.jpeg']

In [157]:
%%time
# get the extracted input features

blk_0303_all_features = []
for file in images_block_0303:
    extracted_features = extract_features(file, block_0303)
    blk_0303_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.9 s, total: 2min 48s
Wall time: 2min 46s


In [158]:
# stack the extracted features
blk_0303_stacked_extracted_features = np.stack(blk_0303_all_features, axis = 1)

In [159]:
blk_0303_stacked_extracted_features.shape

(1376, 7, 32)

In [160]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0303_stacked_extracted_features)

0.16399278

In [161]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0303.npy", blk_0303_stacked_extracted_features)

In [162]:
# sanity check
np.mean(blk_0303_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0303.npy"))

1.0

In [163]:
# Block 0304

In [164]:
block_0304 = "../../../Spring_2024/S_lab_TasselNet/Block_16_TN/Block_16_images_and_xml"

In [165]:
images_block_0304 = chose_xml_and_jpeg(block_0304)

In [166]:
images_block_0304

['Block0304_2020_08_26.jpeg',
 'Block0304_2020_08_27.jpeg',
 'Block0304_2020_08_28.jpeg',
 'Block0304_2020_08_31.jpeg',
 'Block0304_2020_09_02.jpeg',
 'Block0304_2020_09_07.jpeg',
 'Block0304_2020_09_16.jpeg']

In [167]:
%%time
# get the extracted input features

blk_0304_all_features = []
for file in images_block_0304:
    extracted_features = extract_features(file, block_0304)
    blk_0304_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.1 s, total: 2min 48s
Wall time: 2min 45s


In [168]:
# stack the extracted features
blk_0304_stacked_extracted_features = np.stack(blk_0304_all_features, axis = 1)

In [169]:
blk_0304_stacked_extracted_features.shape

(1376, 7, 32)

In [170]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0304_stacked_extracted_features)

0.22966018

In [171]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0304.npy", blk_0304_stacked_extracted_features)

In [172]:
# sanity check
np.mean(blk_0304_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0304.npy"))

1.0

In [173]:
# Block 0305

In [174]:
block_0305 = "../../../Spring_2024/S_lab_TasselNet/Block_17_TN/Block_17_images_and_xml"

In [175]:
images_block_0305 = chose_xml_and_jpeg(block_0305)

In [176]:
images_block_0305

['Block0305_2020_08_26.jpeg',
 'Block0305_2020_08_27.jpeg',
 'Block0305_2020_08_28.jpeg',
 'Block0305_2020_08_31.jpeg',
 'Block0305_2020_09_02.jpeg',
 'Block0305_2020_09_07.jpeg',
 'Block0305_2020_09_16.jpeg']

In [177]:
%%time
# get the extracted input features

blk_0305_all_features = []
for file in images_block_0305:
    extracted_features = extract_features(file, block_0305)
    blk_0305_all_features.append(extracted_features)

(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
CPU times: user 2min 35s, sys: 12.3 s, total: 2min 47s
Wall time: 2min 44s


In [178]:
# stack the extracted features
blk_0305_stacked_extracted_features = np.stack(blk_0305_all_features, axis = 1)

In [179]:
blk_0305_stacked_extracted_features.shape

(1376, 7, 32)

In [180]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0305_stacked_extracted_features)

0.22754368

In [181]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0305.npy", blk_0305_stacked_extracted_features)

In [182]:
# sanity check
np.mean(blk_0305_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0305.npy"))

1.0

In [13]:
# Block 0306

In [14]:
block_0306 = "../../../Spring_2024/S_lab_TasselNet/Block_18_TN/Block_18_images_and_xml"

In [15]:
images_block_0306 = chose_xml_and_jpeg(block_0306)

In [16]:
images_block_0306

['Block0306_2020_08_26.jpeg',
 'Block0306_2020_08_27.jpeg',
 'Block0306_2020_08_28.jpeg',
 'Block0306_2020_08_31.jpeg',
 'Block0306_2020_09_02.jpeg',
 'Block0306_2020_09_07.jpeg',
 'Block0306_2020_09_16.jpeg']

In [17]:
%%time
# get the extracted input features

blk_0306_all_features = []
for file in images_block_0306:
    extracted_features = extract_features(file, block_0306)
    blk_0306_all_features.append(extracted_features)

(1376, 300, 300, 3)


2025-09-08 21:11:21.472060: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


43/43 [==============================] - 4s 17ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 18ms/step
(1376, 300, 300, 3)
43/43 [==============================] - 1s 16ms/step
CPU times: user 2min 33s, sys: 13.2 s, total: 2min 46s
Wall time: 2min 46s


In [18]:
# stack the extracted features
blk_0306_stacked_extracted_features = np.stack(blk_0306_all_features, axis = 1)

In [19]:
blk_0306_stacked_extracted_features.shape

(1376, 7, 32)

In [20]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0306_stacked_extracted_features)

0.2732109

In [21]:
# save the data
np.save("data/test_out_targets/test_targets_blk_0306.npy", blk_0306_stacked_extracted_features)

In [22]:
# sanity check
np.mean(blk_0306_stacked_extracted_features == np.load("data/test_out_targets/test_targets_blk_0306.npy"))

1.0